### Imports

In [ ]:
from LogicalQ.Logical import LogicalCircuit, LogicalStatevector, logical_state_fidelity
from LogicalQ.Library.QECCs import steane_code
from LogicalQ.Library.HardwareModels import hardware_models_Quantinuum
from LogicalQ.Execution import execute_circuits
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit
from qiskit.transpiler import PassManager
from LogicalQ.Transpilation.UnBox import UnBoxTask

### Rx Gate

In [ ]:
qecc = steane_code
n, k, d = qecc['label']
theta = 15.0

circ = QuantumCircuit(1)
circ.rx(theta, 0)

lqcirc = LogicalCircuit(1, **qecc, num_ancillas=1)
lqcirc.encode(0)
lqcirc.rx(theta, 0)
lqcirc.measure([0], [0], with_error_correction=False)

result = execute_circuits(lqcirc, backend="aer_simulator", hardware_model=hardware_models_Quantinuum["H2-1"], coupling_map=None, shots=1, save_statevector=True)[0]

l_sv = result.get_statevector()
lsv = LogicalStatevector(l_sv, 1, qecc["label"], qecc["stabilizer_tableau"], lqcirc, basis="physical")

p_sv = Statevector(circ)

fidelity = logical_state_fidelity(lsv, p_sv)
fidelity
#lqcirc.decompose().draw(output='mpl')

In [ ]:
lsv.draw(output='latex')

In [ ]:
p_sv.draw(output='latex')

### Ry Gate

In [ ]:
qecc = steane_code
n, k, d = qecc['label']

lqcirc = LogicalCircuit(1, **qecc)

theta = 15.0

circ = QuantumCircuit(1)
circ.ry(theta, 0)

lqcirc.encode(0)
lqcirc.ry(theta, 0)
lqcirc.measure([0], [0], with_error_correction=False)
#lqcirc.draw(output='mpl')

In [ ]:
result = execute_circuits(lqcirc, backend="aer_simulator", hardware_model=hardware_models_Quantinuum["H2-1"], coupling_map=None, shots=1, save_statevector=True)[0]

l_sv = result.get_statevector()
lsv = LogicalStatevector(l_sv, lqcirc, k, qecc["label"], qecc["stabilizer_tableau"])

p_sv = Statevector(circ)

fidelity = logical_state_fidelity(lsv, p_sv)
fidelity

In [ ]:
lsv.draw(output='latex')

In [ ]:
p_sv.draw(output='latex')

### Rz Gate

In [30]:
qecc = steane_code
n, k, d = qecc['label']

lqcirc = LogicalCircuit(1, **qecc)

theta = 15.0

circ = QuantumCircuit(1)
circ.h(0)
circ.t(0)
circ.rz(theta, 0)

lqcirc.encode(0)
lqcirc.h(0)
lqcirc.t(0)
lqcirc.rz(theta, 0, depth=20, recursion_degree=1)
lqcirc.measure([0], [0], with_error_correction=False)
#lqcirc.draw(output='mpl')

Warning: Box below will draw an enormous circuit.

In [ ]:
pm = PassManager([UnBoxTask()])
qc_unboxed = pm.run(lqcirc)
qc_unboxed.draw(output='mpl')

In [ ]:
result = execute_circuits(lqcirc, backend="aer_simulator", shots=1000, save_statevector=True)[0]#, hardware_model=hardware_models_Quantinuum["H2-1"], coupling_map=None)[0]

l_sv = result.get_statevector()
lsv = LogicalStatevector(l_sv, 1, qecc["label"], qecc["stabilizer_tableau"], lqcirc, basis="physical")
#counts = result.get_counts()
#lsv = LogicalStatevector.from_counts(counts, k, **qecc)

p_sv = Statevector(circ)

fidelity = logical_state_fidelity(lsv, p_sv)
fidelity

0.9999225475185245

In [ ]:
lsv.draw(output='latex')

In [ ]:
p_sv.draw(output='latex')

### Rxx Gate

LogicalStateVector currently does not have a sufficiently fast implementation to handle LSV construction from the physical statevector for a system with 2 logical qubits. We will revisit testing later.

In [26]:
qecc = steane_code
n, k, d = qecc['label']

lqcirc = LogicalCircuit(2, **qecc)

theta = 15.0

circ = QuantumCircuit(2)
circ.rxx(theta, 0, 1)

lqcirc.encode([0, 1])
lqcirc.rxx(theta, [0, 1])
lqcirc.measure([0, 1], [0, 1], with_error_correction=False)
#lqcirc.draw(output='mpl')

In [ ]:
pm = PassManager([UnBoxTask()])
qc_unboxed = pm.run(lqcirc)

qc_unboxed.draw(output='mpl')

qc_unboxed.count_ops()

In [27]:
result = execute_circuits(lqcirc, backend="aer_simulator", shots=1, save_statevector=True)[0]#, hardware_model=hardware_models_Quantinuum["H2-1"], coupling_map=None)[0]

In [28]:
l_sv = result.get_statevector()

In [29]:
lsv = LogicalStatevector(l_sv, lqcirc, k, qecc["label"], qecc["stabilizer_tableau"])

ValueError: Data basis must be specified for LogicalStatevector construction from a statevector

In [ ]:
p_sv = Statevector(circ)

fidelity = logical_state_fidelity(lsv, p_sv)
fidelity

Notes: Initializing LSV from Statevector is very slow. WIP to fix before Rxx gate validation.